In [28]:
from typing import List

import numpy as np
import pandas as pd
from Clasificador import Clasificador


class Individuo:
    def __init__(self, reglas):
        self.reglas = reglas

    def crea_con_reglas_aleatorias(max_reglas: int, longitud_reglas: int):
        reglas = np.random.randint(
            2, size=(np.random.randint(1, max_reglas), longitud_reglas)
        )
        return Individuo(reglas)


class AlgoritmoGenetico(Clasificador):
    def __init__(self, tamano_poblacion: int, epocas: int, max_reglas: int):
        self.tamano_poblacion = tamano_poblacion
        self.epocas = epocas
        self.max_reglas = max_reglas
        self.poblacion = []

    def entrenamiento(self, datosTrain, nominalAtributos, diccionario):
        self._crea_codificacion(datosTrain, nominalAtributos)
        # Crear primera generacion
        # Obtener tamanio de reglas
        longitud_reglas = self._longitud_reglas(datosTrain)
        # Generar `tamano_poblacion` individuos con reglas aleatorias
        for _ in range(self.tamano_poblacion):
            self.poblacion.append(
                Individuo.crea_con_reglas_aleatorias(self.max_reglas, longitud_reglas)
            )

        # for _ in range(epocas):

    def _longitud_reglas(self, datos: pd.DataFrame) -> int:
        X = datos.iloc[:, :-1]
        return X.nunique().sum() + 1

    def _codificar_datos(self, datos: pd.DataFrame) -> np.ndarray:
        if self.codificacion is None:
            self.crea_codificacion(datos)
        pass

    def _crea_codificacion(self, datos: pd.DataFrame, nominalAtributos: List[bool]):
        self.codificacion = [{}] * len(nominalAtributos)

        for i_atributo, _ in enumerate(nominalAtributos):
            valores = [str(valor) for valor in datos.iloc[:, i_atributo].unique()]
            valores = sorted(valores)

            for i_valor, valor in enumerate(valores):
                codigo = [0] * len(valores)
                codigo[i_valor] = 1

                self.codificacion[i_atributo][valor] = codigo

In [ ]:
from typing import List

import numpy as np
from ManipuladorDatos import ManipuladorDatos

from Clasificador import Clasificador


class ClasificadorKNN(Clasificador, ManipuladorDatos):
    def __init__(self, K: int, normaliza=False):
        """
        Inicializa un clasificador K-Nearest Neighbors (KNN).

        Args:
            K (int): Número de vecinos a considerar para la clasificación.
        """
        self.K = K
        self.normaliza = normaliza

    def entrenamiento(self, datosTrain, nominalAtributos, diccionario):
        """
        Realiza el entrenamiento del clasificador KNN.

        Args:
            datosTrain (pd.DataFrame): DataFrame con los datos de entrenamiento.
            nominalAtributos (List[bool]): Lista de booleanos indicando si los atributos son nominales o no.
            diccionario (dict): Diccionario que mapea nombres de atributos a valores nominales.

        Returns:
            None
        """
        self._X_train = datosTrain.iloc[:, :-1]
        self._y_train = datosTrain.iloc[:, -1]

        self._X_media, self._X_desv = self._calcular_medias_desv(self._X_train)

        if self.normaliza:
            self._X_train = self._normalizar_datos(
                self._X_train, self._X_media, self._X_desv
            )

    def clasifica(self, datosTest, nominalAtributos, diccionario):
        """
        Realiza la clasificación de nuevos datos utilizando el clasificador KNN.

        Args:
            datosTest (pd.DataFrame): DataFrame con los datos de prueba a clasificar.
            nominalAtributos (List[bool]): Lista de booleanos indicando si los atributos son nominales o no.
            diccionario (dict): Diccionario que mapea nombres de atributos a valores nominales.

        Returns:
            np.array: Un arreglo NumPy con las predicciones de clase para los datos de prueba.
        """
        X_test = datosTest.iloc[:, :-1]

        if self.normaliza:
            X_test = self._normalizar_datos(X_test, self._X_media, self._X_desv)

        preds = []
        for i_test, X_test_fila in X_test.iterrows():
            distancias = []

            for i_train, X_train_fila in self._X_train.iterrows():
                dist = self._distancia_euclidea(X_test_fila, X_train_fila)
                distancias.append((dist, self._y_train[i_train]))

            distancias_ordenadas = sorted(distancias)
            clases_k_vecinos = [clase for _, clase in distancias_ordenadas[: self.K]]

            pred = self._clase_mas_frecuente(clases_k_vecinos)
            preds.append(pred)

        return np.array(preds)

    def _clase_mas_frecuente(self, clases: List[int]):
        """
        Determina la clase más frecuente en una lista de clases.

        Args:
            clases (List[int]): Lista de clases de vecinos cercanos.

        Returns:
            int: La clase más frecuente en la lista.
        """
        conteo_clases = {}

        for clase in clases:
            if clase in conteo_clases:
                conteo_clases[clase] += 1
            else:
                conteo_clases[clase] = 1

        clase_mas_frecuente = max(conteo_clases, key=conteo_clases.get)

        return clase_mas_frecuente


In [8]:
import pandas as pd

df = pd.read_csv("titanic.csv")
df_titanic = df

In [6]:
df.iloc[:,:-1].nunique().sum()

21

In [29]:
ag = AlgoritmoGenetico(10, 5, 5)
ag.entrenamiento(df_titanic, [True, False, True], [True, False, True])

In [30]:
ag.codificacion

[{'1': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '2': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  '3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  'female': [1, 0],
  'male': [0, 1],
  '0': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '10': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '11': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '12': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '13': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '14': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '15': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  '4': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  '5': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  '6': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  '7': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  '8': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  '9': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]},
 {'1': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,